In [1]:
from os import listdir,path
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import cv2
cv2.__version__

from scipy.stats import norm
from scipy.ndimage import maximum_filter, minimum_filter
from scipy.ndimage.morphology import binary_fill_holes

In [4]:
def imshow_single(img, title):
    fig,ax = plt.subplots()
    ax.imshow(img,cmap='gray')
    ax.set_title(title)
def imshow_double(img1, img2, title):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(img1,cmap='gray')
    ax2.imshow(img2,cmap='gray')
    fig.suptitle(title, fontsize=16)

In [2]:
from dicomUtils import *
vol_path='../data/PACS/Larry_Smarr_2016/series_23_Cor_LAVA_PRE-Amira/'
vs = getVolume(vol_path)

In [3]:
volume_data = vs.raw_data
volume_data.shape
img = volume_data[:,:,70]
img = img.astype(np.uint16)

In [17]:
def foreground(img):
    try:
        h = ex.equalize_hist(img[:,:])*255
        oi = np.zeros_like(img, dtype=np.uint16)
        oi[(img > threshold_otsu(img)) == True] = 1
        oh = np.zeros_like(img, dtype=np.uint16)
        oh[(h > threshold_otsu(h)) == True] = 1
        nm = img.shape[0] * img.shape[1]
        w1 = np.sum(oi)/(nm)
        w2 = np.sum(oh)/(nm)
        ots = np.zeros_like(img, dtype=np.uint16)
        new =( w1 * img) + (w2 * h)
        ots[(new > threshold_otsu(new)) == True] = 1
        conv_hull = convex_hull_image(ots)
        ch = np.multiply(conv_hull, 1)
        fore_image = ch * img
        back_image = (1 - ch) * img
    except Exception: 
        print("exception")
        fore_image = img.copy()
        back_image = np.zeros_like(img, dtype=np.uint16)
        conv_hull = np.zeros_like(img, dtype=np.uint16)
    return fore_image, back_image, conv_hull, img[conv_hull], img[conv_hull==False]

In [36]:
rimg = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB).astype(np.uint8)

mask = np.zeros(img.shape,np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
rect = (0,0,500,500)
cv2.grabCut(rimg,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
rimg = rimg*mask2[:,:,np.newaxis]
imshow_double(img, rimg, 'ori-fore')

error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/grabcut.cpp:99: error: (-5:Bad argument) _model must have CV_64FC1 type, rows == 1 and cols == 13*componentsCount in function 'GMM'


In [37]:
fore_image, back_image, conv_hull, conv_hull, conv_hall_false = foreground(img)

exception
